# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean


In [2]:
pd.set_option('display.max_columns', None)
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 连接数据库

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [4]:
sql = '''
select 
tprm.order_create_time '下单时间', tprm.search_time '下单月份', tprm.order_number '订单号', om.user_mobile '手机号', tprm.total_receivable '总应收', tprm.all_rental '总租金', tod.new_actual_money '买断价', (tod.new_actual_money-tprm.all_rental) '买断尾款', tprm.commodity_premium '溢价费', tprm.buyout_balance_payment '残值', om.all_sesame_promo_money '总差异化定价优惠金额', tprm.model_number '型号', tprm.specification '规格', tprm.rembursement_status '还款状态', om.rent_grade '租物等级', tprm.paid_periods '已还期数'
, tod.sku_attributes, om.rent_grade 差异化等级, om.status 订单状态
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips 
FROM db_digua_business.t_postlease_receivables_monitoring tprm
LEFT JOIN db_digua_business.t_order om ON tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
where om.rent_grade is not null
and date_format(om.create_time, '%Y-%m-%d') >= '2025-02-11'
and date_format(om.create_time, '%Y-%m-%d') <= '2025-02-16'
'''
df = query(sql)

In [5]:
# df.to_excel('F:/需求/廖志彧需求/差异化定价数据.xlsx', index=False)

In [6]:
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")

In [7]:
def cyh(df):
    df_cy_all = df[['订单号', '总租金', '买断价', '差异化等级']]
    df_cy_group = df.groupby('差异化等级').agg(数量=('订单号', 'count'))
    df_cy_group.loc[:, '总计'] = df_cy_group.数量.sum()
    df_cy_group.loc[:, '比例'] = (df_cy_group.数量/df_cy_group.总计).map(lambda x: format(x, '.2%'))
    return df_cy_all, df_cy_group

In [8]:
df_cy = df[(df.差异化等级.notna())]
df_cy_all, df_cy_group = cyh(df_cy)
df_cy_group

,数量,总计,比例
差异化等级,,,
A,37,228,16.23%
B,52,228,22.81%
C,128,228,56.14%
S,11,228,4.82%


In [9]:
df_cy_jl = df_cy[df_cy.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True]
df_cy_jl_all, df_cy_jl_group = cyh(df_cy_jl)

In [12]:
df_cy_jl_all

,订单号,总租金,买断价,差异化等级
19,A20250211132900116,11697.80,11697.8,C
25,A20250211135034239,12997.70,12997.7,C
42,A20250211163958268,12997.70,12997.7,C
46,A2025021119021014,5794.79,12996.7,C
63,A2025021215020014,12997.70,12997.7,C
80,A20250211114306257,11697.80,11697.8,C
101,A20250213125009246,9098.00,9098.0,C
129,A20250212165934370,12997.70,12997.7,C
138,A20250214095622106,5794.79,12996.7,A
147,A20250214112909130,5794.79,12996.7,B


In [10]:
df_cy_jl_8 = df_cy_jl[df_cy_jl.订单状态==4]
df_cy_jl_8

,下单时间,下单月份,订单号,手机号,总应收,总租金,买断价,买断尾款,溢价费,残值,总差异化定价优惠金额,型号,规格,还款状态,租物等级,已还期数,sku_attributes,差异化等级,订单状态,tips,租赁方案
19,2025-02-11 13:29:00,2025-02,A20250211132900116,18789037843,11697.80,11697.80,11697.8,0.00,0.0,0.0,0.0,iPhone16Pro,内存：256GB 颜色：白色钛金属,未到首期还款日,C,2,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",C,4,"[30天多头>9, GPS最近地点:获取异常, 三地校验不一致:IP:海南三沙、定位:未知未...",租完即送
25,2025-02-11 13:50:35,2025-02,A20250211135034239,15819055240,12997.70,12997.70,12997.7,0.00,0.0,0.0,0.0,iPhone16ProMax,内存：256GB 颜色：黑色钛金属,未到首期还款日,C,2,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",C,4,"[3日内GPS(1000)米内下单人数:3, 7日内GPS(1000)米内下单人数:6, G...",租完即送
42,2025-02-11 16:39:58,2025-02,A20250211163958268,15959906075,12997.70,12997.70,12997.7,0.00,0.0,0.0,0.0,iPhone16ProMax,内存：256GB 颜色：沙漠色钛金属,未到首期还款日,C,2,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",C,4,"[GPS最近地点:获取异常, IP与定位地址校验不一致:IP:福建厦门、定位:未知未知, 免...",租完即送
46,2025-02-11 19:02:11,2025-02,A2025021119021014,18207901098,11794.19,5794.79,12996.7,7201.91,0.0,5999.4,0.0,iPhone16ProMax,内存：256GB 颜色：黑色钛金属,未到首期还款日,C,3,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",C,4,"[免押客户, 已有在租, 回捞策略241205命中(4), 回捞策略241205命中(4)]",租完归还
63,2025-02-12 15:02:00,2025-02,A2025021215020014,18500081180,12997.70,12997.70,12997.7,0.00,0.0,0.0,0.0,iPhone16ProMax,内存：256GB 颜色：原色钛金属,未到首期还款日,C,2,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",C,4,"[3日内GPS(1000)米内下单人数:1, 7日内GPS(1000)米内下单人数:1, G...",租完即送
80,2025-02-11 11:43:06,2025-02,A20250211114306257,19103945520,11697.80,11697.80,11697.8,0.00,0.0,0.0,0.0,iPhone16Pro,内存：256GB 颜色：黑色钛金属,未到首期还款日,C,2,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",C,4,"[免押客户, 强制转租送, 手机三要素验证返回不一致, 回捞策略241205命中(1)]",租完即送
129,2025-02-12 16:59:35,2025-02,A20250212165934370,19918188802,12997.70,12997.70,12997.7,0.00,0.0,0.0,0.0,iPhone16ProMax,内存：256GB 颜色：黑色钛金属,未到首期还款日,C,2,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",C,4,"[30天多头>9, 免押客户, 强制转租送, 回捞策略241205命中(1), 命中Fico...",租完即送
